In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import random
from glob import glob
import os, shutil
from tqdm import tqdm
tqdm.pandas()
import time
import copy
import joblib
from collections import defaultdict
import gc
from IPython import display as ipd


import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
 
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold

import tensorflow as tf
from tensorflow import keras
%env SM_FRAMEWORK=tf.keras

import PIL
from PIL import Image
import cv2
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import fnmatch
def wordMap(x, s, e ):
    start = x.index(s)+len(s)
    end = x.index(e, start)
    subw = x[start:end]
    return subw 

def pathMatch(dir, path):
    files = os.scandir(dir)
    for file in files:
        if fnmatch.fnmatch(file.name, path):
            return dir+"/"+file.name

images_path ="/kaggle/input/uw-madison-gi-tract-image-segmentation/train/"
train_csv_path ="/kaggle/input/uw-madison-gi-tract-image-segmentation/train.csv"

def getProcessedDF(path, mode="train"):
    if mode == "train":
        data = pd.read_csv(path)

        df  = data.pivot_table(values='segmentation',index='id',columns='class',aggfunc='max').reset_index()
        
        df2 = data.drop_duplicates(subset=['id'])

        df3 = pd.DataFrame(columns=['id','large_bowel','small_bowel','stomach'])
        df3['id'] = df2[~df2['id'].isin(df['id'])].id.values
        df = pd.concat([df,df3])
        df.reset_index(inplace=True,drop=True)

        df["id"] = df.id.map(lambda x : x)
        df["case"] = df.id.map(lambda x : int(x.split("_")[0].replace("case", "")))
        df["day"] =  df.id.map(lambda x : wordMap(x,"_day","_"))
        df['slice'] = df.id.map(lambda x :  x.split("_")[-1])
        df["image_path"] = df.id.map(lambda x : images_path+wordMap(x,"","_")+"/"+wordMap(x,"","_")+"_day"+wordMap(x,"_day","_")+"/scans/slice_"+x.split("_")[-1]+"|")
        df["image_path"] = df.image_path.map(lambda x : pathMatch(wordMap(x,"","/slice_"),wordMap(x,"scans/","|")+"*.png"))
        df["width"] = df["image_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[1]))
        df["height"] = df["image_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[2]))
    
        
        df["empty"] = df.id
        for i in range (len(df.id)):
            if str(df["large_bowel"].iloc[i]) == "nan":
                df["large_bowel"].iloc[i] = ""
            if str(df["small_bowel"].iloc[i]) == "nan":
                df["small_bowel"].iloc[i] = ""
            if str(df["stomach"].iloc[i]) == "nan":
                df["stomach"].iloc[i]= ""
            df["empty"].iloc[i] = df["large_bowel"].iloc[i]=="" and df["small_bowel"].iloc[i]=="" and df["stomach"].iloc[i]==""
        df['count'] = np.sum(df.iloc[:,1:4]!='',axis=1).values
    
    else :
        
        df = pd.read_csv(path)
                
        df["id"] = df.id.map(lambda x : x)
        df["case"] = df.id.map(lambda x : int(x.split("_")[0].replace("case", "")))
        df["day"] =  df.id.map(lambda x : wordMap(x,"_day","_"))
        df['slice'] = df.id.map(lambda x :  x.split("_")[-1])
        df["image_path"] = df.id.map(lambda x : images_path+wordMap(x,"","_")+"/"+wordMap(x,"","_")+"_day"+wordMap(x,"_day","_")+"/scans/slice_"+x.split("_")[-1]+"|")
        df["image_path"] = df.image_path.map(lambda x : pathMatch(wordMap(x,"","/slice_"),wordMap(x,"scans/","|")+"*.png"))
        df["width"] = df["image_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[1]))
        df["height"] = df["image_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[2]))
        
    df.sort_values(by=['case','day','slice'],ignore_index=True, inplace=True)
    return df

In [ ]:
df = getProcessedDF(path=train_csv_path, mode="train")
df

In [ ]:
fault1 = 'case7_day0'
fault2 = 'case81_day30'
df = df[~df['id'].str.contains(fault1) & ~df['id'].str.contains(fault2)].reset_index(drop=True)
df.to_csv("prova.csv")
gc.collect()

In [ ]:
df["empty"].value_counts().plot.bar(color = ["blue","red"])

In [ ]:
df.info()

<h4>RLE Function</h4>

In [ ]:
def rle_decode(mask_rle,height, width ):
    mask_rle = [int(k) for k in mask_rle.split()]
    start_index = mask_rle[::2]
    length = mask_rle[1::2]
    start_index = [x-1 for x in start_index]
    end_index = [x + y for x,y in zip(start_index, length)]
    img = np.zeros(height * width)
    for i in range(len(start_index)):
        img[start_index[i]:end_index[i]] = 255
    return img.reshape((height, width)).astype(np.uint8)

# Run-length encoding
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

In [ ]:
example_row = df[df["empty"]==False]
example_row = example_row[example_row["height"]>300].iloc[0,:]
print(example_row)
gc.collect()

In [ ]:
path_img = example_row.image_path
path_img

In [ ]:
import matplotlib.pyplot as plt

def print_img_info(img):
    img_arr = np.asarray(img)
    print(img_arr.shape)
    print(img_arr[100])
    plt.imshow(img, cmap="bone")

In [ ]:
img = Image.open(example_row.image_path)

print_img_info(img)


In [ ]:
mask_large_bowel = example_row.large_bowel
mask_small_bowel = example_row.small_bowel
mask_stomach = example_row.stomach
    

In [ ]:
mask = rle_decode(mask_stomach,width=example_row.width,height=example_row.height)
print(mask.shape)
print(mask)
plt.imshow(mask)

In [ ]:
mask = rle_decode(mask_small_bowel,width=example_row.width,height=example_row.height)
print(mask.shape)
print(mask)
plt.imshow(mask)

In [ ]:
mask = rle_decode(mask_large_bowel,width=example_row.width,height=example_row.height)
print(mask.shape)
print(mask)
plt.imshow(mask)
gc.collect()

In [ ]:
num = 5
segmentation_df_example = df[df["empty"]==False].sample(num)

In [ ]:
import matplotlib.colors as col

fig, ax = plt.subplots(num,5, figsize=(18, 8*num))
colors = ['yellow','green','red']


cmap1 = col.ListedColormap(colors[0])
cmap2 = col.ListedColormap(colors[1])
cmap3= col.ListedColormap(colors[2])

for i in range(num):
    record = segmentation_df_example.iloc[i, :]
    
    img = tf.keras.preprocessing.image.load_img(record.image_path, color_mode = "grayscale")
    img = tf.keras.preprocessing.image.img_to_array(img)
    ax[i, 0].imshow(img[:,:,0], cmap="bone")
    ax[i, 0].set_title(record.id)
    
    large_bowel = rle_decode(record.large_bowel,width=record.width,height=record.height)
    small_bowel = rle_decode(record.small_bowel,width=record.width,height=record.height)
    stomach = rle_decode(record.stomach,width=record.width,height=record.height)
    
    ax[i,1].imshow(large_bowel,cmap="bone") 
    ax[i,1].set_title("Large Bowel Mask")
    
    ax[i, 2].imshow(small_bowel, cmap="bone")
    ax[i,2].set_title("Small Bowel Mask")
    
    ax[i,3].imshow(stomach, cmap="bone")
    ax[i,3].set_title("Stomach Mask")
    
    ax[i,4].imshow(img[:,:,0], cmap="bone")
    if record.large_bowel !="":
        ax[i,4].imshow(np.ma.masked_where(large_bowel==False, large_bowel),cmap=cmap1, alpha=1)
    if record.small_bowel !="":    
        ax[i,4].imshow(np.ma.masked_where(small_bowel==False, small_bowel),cmap=cmap2, alpha=1)
    if record.stomach !="":    
        ax[i,4].imshow(np.ma.masked_where(stomach==False, stomach),cmap=cmap3, alpha=1)
    ax[i,4].set_title("IMG + MASKs")
gc.collect()

In [ ]:
def histoPlot(img):
    hist = plt.hist(img.ravel(), bins=256)

    plt.title("Istogramma")
    plt.xlabel("Intensità dei pixel")
    plt.ylabel("Numero di pixel")
    plt.show()

def preprocess(img_path,width,height ):
    img = Image.open(img_path).resize((width,height))
    img = (img - np.min(img)) * (1 / (np.max(img) - np.min(img)))
    img = np.expand_dims(img, axis=-1)
    return img
    


In [ ]:
sample1 = segmentation_df_example.iloc[0, :]
img_s = Image.open(sample1.image_path)
print_img_info(img_s)
gc.collect()

In [ ]:
img_s =preprocess(sample1.image_path, 128,128)
print_img_info(img_s)
gc.collect()

In [ ]:
import matplotlib.colors as col

const_resize=(128,128)

fig, ax = plt.subplots(num,5, figsize=(18, 8*num))
colors = ['yellow','green','red']


cmap1 = col.ListedColormap(colors[0])
cmap2 = col.ListedColormap(colors[1])
cmap3= col.ListedColormap(colors[2])

for i in range(num):
    record = segmentation_df_example.iloc[i, :]
    
    img = preprocess(record.image_path,128,128)
    
    ax[i, 0].imshow(img, cmap="bone")
    ax[i, 0].set_title(record.id)
    
    large_bowel = rle_decode(record.large_bowel,width=record.width,height=record.height)
    small_bowel = rle_decode(record.small_bowel,width=record.width,height=record.height)
    stomach = rle_decode(record.stomach,width=record.width,height=record.height)
    
    large_bowel = np.array(Image.fromarray(large_bowel).resize(const_resize))
    large_bowel = (mask == 255).astype(np.float32)
    
    small_bowel = np.array(Image.fromarray(small_bowel).resize(const_resize))
    small_bowel = (small_bowel == 255).astype(np.float32)
    
    stomach = np.array(Image.fromarray(stomach).resize(const_resize))
    stomach = (stomach == 255).astype(np.float32)
    
    
    ax[i,1].imshow(large_bowel,cmap="bone") 
    ax[i,1].set_title("Large Bowel Mask")
    
    ax[i, 2].imshow(small_bowel, cmap="bone")
    ax[i,2].set_title("Small Bowel Mask")
    
    ax[i,3].imshow(stomach, cmap="bone")
    ax[i,3].set_title("Stomach Mask")
    
    ax[i,4].imshow(img, cmap="bone")
    if record.large_bowel !="":
        ax[i,4].imshow(np.ma.masked_where(large_bowel==False, large_bowel),cmap=cmap1, alpha=1)
    if record.small_bowel !="":    
        ax[i,4].imshow(np.ma.masked_where(small_bowel==False, small_bowel),cmap=cmap2, alpha=1)
    if record.stomach !="":    
        ax[i,4].imshow(np.ma.masked_where(stomach==False, stomach),cmap=cmap3, alpha=1)
    ax[i,4].set_title("IMG + MASKs")
gc.collect()

In [ ]:
BATCH_SIZE = 64
EPOCHS = 40
n_spilts = 5
fold_selected = 2
IMG_SIZE = 128
classes = ["large_bowel","small_bowel","stomach"]

In [ ]:

from tensorflow import keras
from keras.utils import Sequence
from PIL import Image
import numpy as np
import tensorflow as tf
import cv2



class DataGenerator(Sequence):
    def __init__(self, df, batch_size=BATCH_SIZE, width=IMG_SIZE, height= IMG_SIZE, mode="train") :
        self.df = df
        self.batch_size = batch_size
        self.width = width
        self.height = height
        self.indexes = np.arange(len(df))
        self.mode = mode
        
    def __len__(self):
        return int(len(self.df) // self.batch_size)
    
    def __load_image(self, img_path):
        img = preprocess(img_path, width=self.width, height=self.height)
        return img.astype(np.float32)
        
    
    
    def __getitem__(self, index):
        
        X = np.empty((self.batch_size,self.height,self.width,3))
        y = np.empty((self.batch_size,self.height,self.width,3))

        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        for i, img_path in enumerate(self.df['image_path'].iloc[indexes]):
            # Load the image
        
            image = self.__load_image(img_path)
            w=self.df['width'].iloc[indexes[i]]
            h=self.df['height'].iloc[indexes[i]]
            X[i,] = image
            
            if self.mode=="train":
                for k,j in enumerate(["stomach", "large_bowel","small_bowel"]):
                    rle = self.df[j].iloc[indexes[i]]
                    mask = rle_decode(rle,height=h,width=w)
                    mask = np.array(Image.fromarray(mask).resize((self.height,self.width)))
                    mask = (mask == 255).astype(np.float32)

                    y[i,:,:,k] = mask
        
        if self.mode =="train":        
            return (X, y)
        
        return X

In [ ]:
import keras.backend as K
from keras.losses import binary_crossentropy
import tensorflow as tf

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_coef(y_true, y_pred, threshold=0.5):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_pred_f = K.cast(K.greater_equal(y_pred_f, threshold), 'float32')
    intersection = K.sum(y_true_f * y_pred_f)
    dice_coef = (2 * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)
    return dice_coef

def dice_coef_loss(y_true, y_pred, epsilon=1e-5):
    return 1 - (2.0 * K.sum(y_true * y_pred) + epsilon) / (K.sum(y_true) + K.sum(y_pred) + epsilon)

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(tf.cast(y_true, tf.float32), y_pred) + dice_coef_loss(tf.cast(y_true, tf.float32), y_pred)


In [ ]:
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(df, test_size = 0.25, shuffle = True, random_state = 42)

In [ ]:
train_set.info()

In [ ]:
val_set.info()

In [ ]:
train_generator = DataGenerator(train_set)
val_generator = DataGenerator(val_set)

In [ ]:
! pip install segmentation-models 

In [ ]:
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()


In [ ]:
from segmentation_models import Unet
from segmentation_models.utils import set_trainable

model = Unet('resnet34',input_shape=(IMG_SIZE, IMG_SIZE, 3), classes=3, activation='sigmoid', encoder_weights='imagenet')
model.compile(optimizer='adam', loss=dice_coef_loss, metrics=[dice_coef])

In [ ]:
model.summary()

In [ ]:
gc.collect()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau,ModelCheckpoint

checkpoint = ModelCheckpoint(
    'UNET_model',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
)

early_stopping = EarlyStopping(
    patience=5,
    min_delta=0.0001,
    restore_best_weights=True,
)

lr_plateau = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=5,
    verbose=0,
    min_delta=0.0001,
)
model.save('final_model')

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    callbacks=[checkpoint, 
               early_stopping],
    use_multiprocessing=False,
    workers=4,
    epochs=EPOCHS
)

In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df.to_csv('history.csv')

plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.plot(range(history.epoch[-1]+1),history.history['loss'],label='Train_Loss')
plt.plot(range(history.epoch[-1]+1),history.history['val_loss'],label='Val_loss')
plt.title('LOSS'); plt.xlabel('Epoch'); plt.ylabel('loss');plt.legend();

plt.subplot(1,3,2)
plt.plot(range(history.epoch[-1]+1),history.history['dice_coef'],label='Train_dice_coef')
plt.plot(range(history.epoch[-1]+1),history.history['val_dice_coef'],label='Val_dice_coef')
plt.title('DICE'); plt.xlabel('Epoch'); plt.ylabel('dice_coef');plt.legend(); 

plt.show()

Mostro alcune Predizioni

In [ ]:
import matplotlib.colors as col

pred_batches = DataGenerator(val_set.iloc[203:208,:], batch_size = 1)
preds = model.predict_generator(pred_batches,verbose=1)

Threshold = 0.5
fig, ax = plt.subplots(num,3, figsize=(18, 8*num))
colors = ['yellow','green','red']
labels = ["Large Bowel", "Small Bowel", "Stomach"]


cmap1 = col.ListedColormap(colors[0])
cmap2 = col.ListedColormap(colors[1])
cmap3= col.ListedColormap(colors[2])

for i in range(5):
    images, mask = pred_batches[i]
    sample_img=images[0,:,:,0]
    mask1=mask[0,:,:,0]
    mask2=mask[0,:,:,1]
    mask3=mask[0,:,:,2]
    
    pre=preds[i]
    predict1=pre[:,:,0]
    predict2=pre[:,:,1]
    predict3=pre[:,:,2]
    
    predict1= (predict1 > Threshold).astype(np.float32)
    predict2= (predict2 > Threshold).astype(np.float32)
    predict3= (predict3 > Threshold).astype(np.float32)
    
    ax[i,0].imshow(sample_img, cmap="bone")
    ax[i,0].set_title("Image", fontsize=12, y=1.01)
    #--------------------------
    ax[i,1].imshow(sample_img, cmap="bone")
    ax[i,1].set_title("Mask", fontsize=12,  y=1.01)
    l0 = ax[i,1].imshow(sample_img, cmap="bone")
    l1 = ax[i,1].imshow(np.ma.masked_where(mask1== False,  mask1),cmap=cmap1, alpha=1)
    l2 = ax[i,1].imshow(np.ma.masked_where(mask2== False,  mask2),cmap=cmap2, alpha=1)
    l3 = ax[i,1].imshow(np.ma.masked_where(mask3== False,  mask3),cmap=cmap3, alpha=1)
    #--------------------------
    
    ax[i,2].set_title("Predict", fontsize=12, y=1.01)
    l0 =ax[i,2].imshow(sample_img, cmap='bone')
    l1 = ax[i,2].imshow(np.ma.masked_where(predict1== False,  predict1),cmap=cmap1, alpha=1)
    l2 = ax[i,2].imshow(np.ma.masked_where(predict2== False,  predict2),cmap=cmap2, alpha=1)
    l3 = ax[i,2].imshow(np.ma.masked_where(predict3== False,  predict3),cmap=cmap3, alpha=1)
   

In [ ]:
test_dir ="/kaggle/input/uw-madison-gi-tract-image-segmentation/sample_submission.csv"
train_df =pd.read_csv("/kaggle/input/uw-madison-gi-tract-image-segmentation/train.csv")
train_df
test_df =  getProcessedDF(path=test_dir, mode="test")


if len(test_df) == 0:
    test_df = df.iloc[:10*16*3,:]
    test_df = test_df.drop(columns=["large_bowel","small_bowel","stomach","empty","count"])
test_df["class"] = 

In [ ]:
test_df.head()

In [ ]:
from keras.models import load_model

custom_objects = custom_objects={
    'dice_coef': dice_coef,
    'dice_coef_loss': dice_coef_loss
}

model_path = "/kaggle/working/UNET_model"
model = load_model(model_path,custom_objects=custom_objects)

In [ ]:
from PIL import Image
pred_batches = DataGenerator(test_df, batch_size = BATCH_SIZE, mode="test")
num_batches = int(len(test_df)/BATCH_SIZE)
submission = pd.read_csv("/kaggle/input/uw-madison-gi-tract-image-segmentation/sample_submission.csv")
for i in range(num_batches):
    preds = model.predict(pred_batches[i],verbose=0)   
 
    for j in range(BATCH_SIZE):
        for k in range(3):
            classes = {0:"large_bowel",1:"small_bowel",2:"stomach"}
            pred_img = cv2.resize(preds[j,:,:,k], (test_df.loc[i*BATCH_SIZE+j,"width"], test_df.loc[i*BATCH_SIZE+j,"height"]),interpolation=cv2.INTER_NEAREST) 
            pred_img = (pred_img>0.5).astype(dtype='uint8')    # classify
            submission.loc[3*(i*BATCH_SIZE+j)+k,'predicted'] = rle_encode(pred_img)
            submission.loc[3*(i*BATCH_SIZE+j)+k,'id'] = test_df.loc[i*BATCH_SIZE+j,"id"]
            submission.loc[3*(i*BATCH_SIZE+j)+k,'class'] = classes.get(k)

In [ ]:
submission.to_csv('submission.csv',index=False)
submission.head()